# Dadbot: Dad's memorial bot based on RASA (old style)

## Starting Jupyter Notebook with necessary imports

In [33]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')


# Installations
* Rasa
* SpaCy Language Model

In [34]:
import sys
python = sys.executable

# In your environment run:
#!{python} -m pip install -U pip
#!{python} -m pip install tensorflow[tensorflow-addons]==2.1.0
#!{python} -m pip install rasa[convert]~=1.10.0

In [35]:
!{python} -m spacy download es_core_news_md

You should consider upgrading via the '/home/debian/.conda/envs/rasa/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


## Downloading the Spanish Language Model

In [36]:
!{python} -m spacy link es_core_news_md es --force;

✔ Linking successful
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/es_core_news_md -->
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


## Import the Installations

In [37]:
import rasa


# 1. Teaching the bot to understand user inputs using Rasa NLU

## Training the NLU Model.

In [38]:
from rasa.model import get_model
from rasa.nlu import config, training_data, utils
from rasa.nlu.utils import write_to_file
from rasa.nlu.components import ComponentBuilder
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Interpreter, Trainer, TrainingData
from rasa.nlu.components import Component
from rasa.nlu.tokenizers.tokenizer import Token
from rasa.utils.tensorflow.constants import ENTITY_RECOGNITION

import spacy

#spacy_parser = spacy.load('es_core_news_md')
#nlp = spacy.load('es')

# loading the nlu training samples
training_data = training_data.loading.load_data("data/nlu/nlu-papaito.md")

# trainer to train our pipeline
trainer = Trainer(config.load("config_simple.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa.nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component RegexFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EmbeddingIntentClassifier
Epochs: 100%|██████████| 300/300 [00:35<00:00,  8.36it/s, t_loss=1.127, i_loss=0.112, i_acc=0.992]
INFO:rasa.utils.tensorflow.models:Finished training.
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Su

## Evaluating the NLU model on a random text

In [39]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("dejándome el coche"))

{
  "intent": {
    "name": "ask_help",
    "confidence": 0.961948037147522
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "ask_help",
      "confidence": 0.961948037147522
    },
    {
      "name": "inform_estado",
      "confidence": 0.01379871740937233
    },
    {
      "name": "ask_deporte_futbol",
      "confidence": 0.005939386319369078
    },
    {
      "name": "ask_vivienda",
      "confidence": 0.003995127975940704
    },
    {
      "name": "ask_cuenta_algo",
      "confidence": 0.0036021017003804445
    },
    {
      "name": "ask_pais",
      "confidence": 0.002997576491907239
    },
    {
      "name": "mood_deny",
      "confidence": 0.0024529730435460806
    },
    {
      "name": "ask_nombre",
      "confidence": 0.0020948059391230345
    },
    {
      "name": "goodbye",
      "confidence": 0.0019319221610203385
    },
    {
      "name": "ask_deporte_bici",
      "confidence": 0.0012393814977258444
    }
  ],
  "text": "dej\u00e1ndome el coche"
}


## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [40]:
from rasa.nlu.test import run_evaluation

run_evaluation("data/nlu/nlu-papaito.md", model_directory)

INFO:rasa.nlu.test:Running model for predictions:
100%|██████████| 612/612 [00:01<00:00, 361.63it/s]
INFO:rasa.nlu.test:Intent evaluation results:
INFO:rasa.nlu.test:Intent Evaluation: Only considering those 612 examples that have a defined intent out of 612 examples
INFO:rasa.nlu.test:F1-Score:  0.9885429120500552
INFO:rasa.nlu.test:Precision: 0.9887066365007543
INFO:rasa.nlu.test:Accuracy:  0.988562091503268
INFO:rasa.nlu.test:Classification report: 
                      precision    recall  f1-score   support

    ask_deporte_bici       1.00      1.00      1.00         9
          ask_estado       1.00      1.00      1.00        31
        ask_vivienda       1.00      1.00      1.00        13
        mood_unhappy       0.99      0.99      0.99        87
       inform_estado       1.00      1.00      1.00        10
         mood_affirm       1.00      1.00      1.00        42
          mood_happy       1.00      0.98      0.99        51
            ask_pais       1.00      1.00     

ntent': 'ask_help',
    'predicted': 'ask_help',
    'confidence': 0.9768481850624084},
   {'text': 'prestándome el coche',
    'intent': 'ask_help',
    'predicted': 'ask_help',
    'confidence': 0.995529294013977},
   {'text': 'dejandome el coche',
    'intent': 'ask_help',
    'predicted': 'ask_help',
    'confidence': 0.9930726885795593},
   {'text': '¿por qué?',
    'intent': 'ask_razon',
    'predicted': 'ask_razon',
    'confidence': 0.9968084096908569},
   {'text': '¿por qué no?',
    'intent': 'ask_razon',
    'predicted': 'ask_razon',
    'confidence': 0.9927771091461182},
   {'text': '¿por?',
    'intent': 'ask_razon',
    'predicted': 'ask_razon',
    'confidence': 0.9567881226539612},
   {'text': 'dime por qué',
    'intent': 'ask_razon',
    'predicted': 'ask_razon',
    'confidence': 0.9789391160011292},
   {'text': 'dime el porqué',
    'intent': 'ask_razon',
    'predicted': 'ask_razon',
    'confidence': 0.9872432947158813},
   {'text': 'quiero que me lo cuentes',
   

# 2. Teaching the bot to respond using Rasa Core

##  Visualising the Training Data

In [41]:

#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!breq install graphviz

#!conda install -y -n rasa pygraphviz pkg-config;

In [42]:
from IPython.display import Image
from rasa.core.agent import Agent
 
agent = Agent('domain-papaito.yml')
#agent.visualize("data/core/stories-papaito.md", "story_graph.png", max_history=2)
#Image(filename="story_graph.png")

## Training a Dialogue Model

In [52]:
from rasa.core.policies.policy import Policy
from rasa.core.policies.registry import FallbackPolicy, KerasPolicy, MemoizationPolicy, FormPolicy, MappingPolicy
from rasa.core.agent import Agent

import asyncio

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain-papaito.yml',
              policies=[MappingPolicy(),
                        MemoizationPolicy(max_history=5),
                        KerasPolicy(validation_split=0.1,epochs=400),
                        FormPolicy(),
                        fallback])

# loading our neatly defined training dialogues
training_data = await agent.load_data('data/core/stories-papaito.md')

agent.train(training_data)

agent.persist('models/dialogue')

curacy: 0.9364
Epoch 219/400
1430/1430 [==============================] - 0s 144us/sample - loss: 0.2006 - accuracy: 0.9273
Epoch 220/400
1430/1430 [==============================] - 0s 122us/sample - loss: 0.2182 - accuracy: 0.9175
Epoch 221/400
1430/1430 [==============================] - 0s 198us/sample - loss: 0.1901 - accuracy: 0.9308
Epoch 222/400
1430/1430 [==============================] - 0s 149us/sample - loss: 0.1636 - accuracy: 0.9427
Epoch 223/400
1430/1430 [==============================] - 0s 133us/sample - loss: 0.2053 - accuracy: 0.9259
Epoch 224/400
1430/1430 [==============================] - 0s 135us/sample - loss: 0.2114 - accuracy: 0.9273
Epoch 225/400
1430/1430 [==============================] - 0s 129us/sample - loss: 0.2009 - accuracy: 0.9308
Epoch 226/400
1430/1430 [==============================] - 0s 137us/sample - loss: 0.2305 - accuracy: 0.9203
Epoch 227/400
1430/1430 [==============================] - 0s 137us/sample - loss: 0.2031 - accuracy: 0.9308
Epoc

TypeError: object NoneType can't be used in 'await' expression

# Talk to your Bot

In [44]:
#Starting the Bot
from rasa.core.agent import Agent
from rasa.core.utils import EndpointConfig

action_endpoint = EndpointConfig(url="http://0.0.0.0:5055/webhook")
agent = Agent.load('models/dialogue', interpreter=model_directory, action_endpoint=action_endpoint)

#!docker run -d -p 5055:5055 --mount type=bind,source=/home/debian/workspace/Dadbot/actions/actions.py,target=/app/actions rasa/rasa-sdk

ModelNotFound: No NLU or Core data for unpacked model at: 'models/dialogue'.

## Creating the voice synthesizer

In [45]:
#!git clone https://github.com/NVIDIA/tacotron2.git

from tacotron2.hparams import create_hparams
from tacotron2.model import Tacotron2
from tacotron2.layers import TacotronSTFT, STFT
from tacotron2.audio_processing import griffin_lim
from tacotron2.train import load_model
from tacotron2.text import text_to_sequence
from tacotron2.waveglow.mel2samp import files_to_list, MAX_WAV_VALUE
from tacotron2.denoiser import Denoiser
import numpy as np
import torch

def synthesize(text, voice, sigma=0.6, denoiser_strength=0.05, is_fp16=False):

    hparams = create_hparams()
    hparams.sampling_rate = 22050

    if voice == "papaito":
        voice_model = "nvidia_tacotron2_papaito_300"
    elif voice == "constantino":
        voice_model = "tacotron2_Constantino_600"
    elif voice == "orador":
        voice_model = "checkpoint_tacotron2_29000_es"
    
    checkpoint_path = "/home/debian/workspace/models/" + voice_model
        
    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval().half()

    waveglow_path = '/home/debian/workspace/models/waveglow_256channels_ljs_v2.pt'
    waveglow = torch.load(waveglow_path)['model']
    _ = waveglow.cuda().eval().half()
    denoiser = Denoiser(waveglow)

    #text="¡Cágate lorito!"
    #with open(filelist_path, encoding='utf-8', mode='r') as f:
    #    text = f.read()

    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).cuda().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    #mel = torch.unsqueeze(mel, 0)
    mel = mel_outputs.half() if is_fp16 else mel_outputs
    audio = np.array([])
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=sigma)
        if denoiser_strength > 0:
             audio = denoiser(audio, denoiser_strength)
        audio = audio * MAX_WAV_VALUE
        audio = audio.squeeze()
        audio = audio.cpu().numpy()
        audio = audio.astype('int16')
    
    return audio, hparams.sampling_rate

ModuleNotFoundError: No module named 'numba.decorators'

## Start chat

In [46]:
import sounddevice as sd
from sty import fg, bg, ef, rs

print("¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'")
while True:
    a = input()
    
    if a == 'quieto parao':
        break
    responses = agent.handle_text(a)
    for response in responses:
        to_synth = response["text"]
        #to_synth = "Esto es una prueba para ver si funciona"
        print(fg.blue + to_synth + fg.rs)
        response_file = open('response.txt','w') 
        response_file.write(to_synth)
        voice, sr = synthesize(to_synth, "orador")
        sd.play(voice, sr)
        response_file.close()        

¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'


KeyboardInterrupt: 